# Reacher


In [1]:
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from unityagents import UnityEnvironment

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
env = UnityEnvironment(file_name='./Reacher_Windows_x86_64/Reacher.exe')
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


### Method to plot the progress of the agent's score

In [3]:
def plot_result(scores):
    # plot the scores
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

# Done
if average among 20 agents > 30 for the last 100 episodes

In [40]:
def train(agent, n_episodes=2000, max_t=500):
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        agent.reset()
        states = env_info.vector_observations
        scores_one_episode = np.zeros(states.shape[0])
        for t in range(max_t):
            actions = agent.act(states)                              # select an action (for each agent)
            env_info = env.step(actions)[brain_name]                 # send all actions to tne environment
            next_states = env_info.vector_observations               # get next state (for each agent)
            rewards = env_info.rewards                               # get reward (for each agent)
            dones = env_info.local_done                              # see if episode finished
            scores_one_episode += env_info.rewards                   # update the score (for each agent)
            agent.step(states, actions, rewards, next_states, dones) # learn
            states = next_states                                     # roll over states to next time step
            if np.any(dones):                                        # exit loop if episode finished
                break
        score = np.average(scores_one_episode)
        scores.append(score)
        scores_window.append(score)
        mean_100 = np.mean(scores_window)
        print('\rEpisode {}\tAverage Score: {:.3f}\tLast Score: {:.3f}'.format(i_episode, mean_100, score), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.3f}\tMax Score: {:.3f}'.format(i_episode, mean_100, np.max(scores_window)))
            agent.save()
        if np.mean(scores_window)>=13.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.3f}'.format(i_episode, mean_100))
            agent.save()
            break
    return scores

In [41]:
from agents.utils import ReplayBuffer
from agents.parallel_ddpg import ParallelDDPG
from agents.models import Actor, Critic

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
random_seed = 10
states = env_info.vector_observations
action_size = brain.vector_action_space_size
memory = ReplayBuffer(action_size, device, int(1e5), 128, random_seed)
agent = ParallelDDPG(states.shape[1], action_size, random_seed, states.shape[0], memory, Actor, Critic, device)
scores = train(agent, max_t=100)
plot_result(scores)

I:\MyDev\Anaconda3\envs\drlnd\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 3	Average Score: 0.038	Last Score: 0.034

KeyboardInterrupt: 